In [1]:
import pandas as pd

In [2]:
def read_vocab(vocab_file):
    """读取词汇表"""
    # words = open_file(vocab_dir).read().strip().split('\n')
    with open_file(vocab_file) as fp:
        # 如果是py2 则每个值都转化为unicode
        words = [_.strip() for _ in fp.readlines()]
        print(len(words))
    word_to_id = dict(zip(words, range(len(words))))
    return words, word_to_id


def open_file(filename, mode='r'):
    return open(filename, mode, encoding='utf-8', errors='ignore')


def read_vocab(vocab_file):
    """读取词汇表"""
    # words = open_file(vocab_dir).read().strip().split('\n')
    with open_file(vocab_file) as fp:
        # 如果是py2 则每个值都转化为unicode
        words = [_.strip() for _ in fp.readlines()]
        print(len(words))
    word_to_id = dict(zip(words, range(len(words))))
    return words, word_to_id


def encode_cate(content, words):
    """将id表示的内容转换为文字"""
    return [(words[x] if x in words else 208408) for x in content.split(" ")]

def encode_sentences(contents, words):
    """将id表示的内容转换为文字"""
    return [encode_cate(x, words) for x in contents]

In [3]:
df_data = pd.read_csv("../project/data/train.csv", sep="###__###", encoding="utf-8", header=None)
df_data.head()
df_data.drop(df_data[df_data[3] == 0].index, inplace=True)

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [4]:
df_data.replace({"http.*?\t": ""}, regex=True, inplace=True)

In [5]:
df_data.replace({"[0-9a-zA-Z]+\t": ""}, regex=True, inplace=True)

In [6]:
df_new = df_data.loc[:,[3, 4]]

In [7]:
stopwords=pd.read_csv("../project/data/stopwords.txt",index_col=False,quoting=3,sep="\t",names=['stopword'], encoding='utf-8')
stopwords=stopwords['stopword'].values

In [ ]:
import jieba
sentences = []
words = []
block_max_len = 3
content_lines = df_new[4]
category_lines = df_new[1]
i = 0
for line in content_lines:
    if i % 5000 == 0:
        print(i)
    seg_list = list(jieba.lcut(line))
    seg_list = list(filter(lambda x:len(x)>1, seg_list))
    filtered_seg_list = list(filter(lambda x:x not in stopwords, seg_list))
    sentences.append((" ".join(filtered_seg_list), category_lines[content_lines[content_lines.values == line].index].values[0]))
    words.extend(filtered_seg_list)
    i += 1

In [ ]:
from collections import Counter
counter = Counter(words)
words = [w[0] for w in counter.most_common() if int(w[1]) > 5]
words = ['<PAD>'] + list(words)
open_file("../project/data/vocab.file", mode='w').write('\n'.join(words) + '\n')

In [ ]:
sentences_str = [str(x) for x in sentences]
open_file("../project/data/sentences.file", mode='w').write('\n'.join(sentences_str) + '\n')

In [8]:
train_data_from_file = []
with open("../project/data/sentences.file", mode="r", encoding="utf-8") as f:
    for line in f.readlines():
        data_tuple = (line[2:-6], int(line[-3]))
        train_data_from_file.append(data_tuple)
    f.close()

In [9]:
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Embedding, Dense, Conv1D, GlobalMaxPooling1D, Concatenate, Dropout


class TextCNN(object):
    def __init__(self, maxlen, max_features, embedding_dims,
                 class_num=7,
                 last_activation='softmax'):
        self.maxlen = maxlen
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.class_num = class_num
        self.last_activation = last_activation

    def get_model(self):
        input = Input((self.maxlen,))
        embedding = Embedding(self.max_features, self.embedding_dims, input_length=self.maxlen)(input)
        convs = []
        for kernel_size in [3, 4, 5]:
            c = Conv1D(128, kernel_size, activation='relu')(embedding)
            c = GlobalMaxPooling1D()(c)
            convs.append(c)
        x = Concatenate()(convs)

        output = Dense(self.class_num, activation=self.last_activation)(x)
        model = Model(inputs=input, outputs=output)
        return model

D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [10]:
import random
from sklearn.model_selection import train_test_split

sentences = train_data_from_file
random.shuffle(sentences)
df_all_data = pd.DataFrame(sentences)
train_data, test_data = train_test_split(df_all_data, stratify=df_all_data[1], random_state=65)

In [11]:
# 路径等配置
vocab_file = "../project/data/vocab.file"
# with open("../project/data/vocab.file", mode="r", encoding="utf-8") as f:
#     print(len(f.readlines()))
#     f.close()
vocab_size = 208408

# 神经网络配置
max_features = 208408 + 1
maxlen = 300
batch_size = 128
embedding_dims = 120
epochs = 8

In [12]:
words, word_to_id = read_vocab(vocab_file)

208408


In [13]:
x_train = encode_sentences([content for content in train_data[0]], word_to_id)
x_test = encode_sentences([content for content in test_data[0]], word_to_id)

In [14]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical([content for content in train_data[1]])
y_test = to_categorical([content for content in test_data[1]])

In [15]:
from tensorflow.keras.preprocessing import sequence
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [16]:
model = TextCNN(maxlen, max_features, embedding_dims).get_model()
model.compile('adam', 'categorical_crossentropy', metrics=['acc'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [17]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
my_callbacks = [
    ModelCheckpoint('../project/model/cnn_model_query.h5', verbose=1),
    EarlyStopping(monitor='val_acc', patience=2, mode='max')
]

In [ ]:
history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=my_callbacks,
          validation_data=(x_test, y_test)) 

Train on 73750 samples, validate on 24584 samples
Epoch 1/8
71808/73750 [============================>.] - ETA: 13s - loss: 1.1675 - acc: 0.5341